In [ ]:
import numpy as np
import cv2
import supervision as sv
from segment_anything import SamPredictor, sam_model_registry
from grounding_dino_demo import detect_with_dino


from const import *
img_path = 'assets/61.jpg'
text_prompt = ", ".join( ['cap', 't shirt', 'sunglasses', 'headband', 'shoe', 'sock', 'backpack', 'walking-sticks', 'bib numbers', 'trousers'])



annotated_frame, boxes,phrases = detect_with_dino(img_path, text_prompt)
cv2.imwrite('assets/61_dino.jpg', annotated_frame)
print (phrases)

In [ ]:
from GroundingDINO.groundingdino.util import box_ops
from PIL import Image
def segment(image, sam_model, boxes):
  sam_model.set_image(image)
  H, W, _ = image.shape
  boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

  transformed_boxes = sam_model.transform.apply_boxes_torch(boxes_xyxy.to(DEVICE), image.shape[:2])
  masks, _, _ = sam_model.predict_torch(
      point_coords = None,
      point_labels = None,
      boxes = transformed_boxes,
      multimask_output = False,
      )
  return masks.cpu()
  

def draw_mask(mask, image, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.8])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    
    annotated_frame_pil = Image.fromarray(image).convert("RGBA")
    mask_image_pil = Image.fromarray((mask_image.cpu().numpy() * 255).astype(np.uint8)).convert("RGBA")


    return np.array(Image.alpha_composite(annotated_frame_pil, mask_image_pil))

In [ ]:
import numpy as np
from PIL import Image
sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_WEIGHTS_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)
image = cv2.imread(img_path)

# Assurez-vous que les bibliothèques et les fonctions requises sont importées et définies avant d'exécuter ce code.


def draw_masks_without_boxes(image, sam_model, boxes):
    masks = segment(image, sam_model, boxes)
    masks_without_boxes = np.zeros_like(image)  # Créer une image vide pour dessiner les masques sans les boîtes
    
    for mask in masks:
        masks_without_boxes = draw_mask(mask, masks_without_boxes, random_color=True)  # Dessiner chaque masque
    
    return masks_without_boxes

# Utilisation :
# Remplacez les valeurs suivantes par vos propres valeurs


# sam_model = Votre modèle
# boxes = Vos boîtes

# Appel de la fonction pour dessiner les masques sans les boîtes
result_image = draw_masks_without_boxes(image, sam_predictor, boxes)

# Afficher l'image résultante
cv2.imwrite('assets/61_sam.jpg', result_image)


In [ ]:
import numpy as np
import torch
from PIL import Image

# Redéfinition de la fonction draw_mask pour dessiner les masques directement sur l'image originale
def draw_mask_on_image(mask, image, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.8])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    
    mask = mask.detach().cpu().numpy()  # Convertir le tenseur en tableau NumPy
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    
    mask_image_pil = Image.fromarray((mask_image * 255).astype(np.uint8)).convert("RGBA")
    image_pil = Image.fromarray(image).convert("RGBA")

    # Superposer le masque sur l'image originale
    final_image = Image.alpha_composite(image_pil, mask_image_pil)

    return np.array(final_image)

def draw_masks_on_original_image(image, sam_model, boxes):
    masks = segment(image, sam_model, boxes)
    masks_list = []  # Liste pour stocker les masques générés
    
    for mask in masks:
        image = draw_mask_on_image(mask, image, random_color=True)  # Dessiner chaque masque sur l'image originale
        masks_list.append(mask.cpu().numpy())
    return image, masks_list

# Utilisation :
# Remplacez les valeurs suivantes par vos propres valeurs
# image = Votre image
# sam_model = Votre modèle
# boxes = Vos boîtes




# Appel de la fonction pour dessiner les masques sur l'image originale
result_image_with_masks, masks = draw_masks_on_original_image(image, sam_predictor, boxes)





# Afficher l'image résultante
cv2.imwrite('assets/61_sam_on_image.jpg', result_image_with_masks)


In [ ]:
box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()
annotated_image = mask_annotator.annotate(scene=image.copy(),)
#annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

sv.plot_image(annotated_image, (16, 16))

In [ ]:
import matplotlib.pyplot as plt

def display_masks(masks_list):
    num_masks = len(masks_list)

    plt.figure(figsize=(10, 6))
    for i in range(num_masks):
        mask = masks_list[i][0] if len(masks_list[i].shape) == 3 else masks_list[i]  # Vérifier la forme du masque
        plt.subplot(1, num_masks, i + 1)
        plt.imshow(mask, cmap='gray')  # Afficher en niveaux de gris si le masque est 2D
        plt.title(f"Masque {i+1}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Utilisation :
# Remplacez masks_retrieved par votre liste de masques récupérés


display_masks(masks)


In [ ]:
mask = result_image_with_masks.cpu().numpy()
inverted_mask = ((1 - mask) * 255).astype(np.uint8)


image_source_pil = Image.fromarray(image)
image_mask_pil = Image.fromarray(mask)
inverted_image_mask_pil = Image.fromarray(inverted_mask)


display(*[image_source_pil, image_mask_pil, inverted_image_mask_pil])

In [ ]:
from groundingdino.util.inference import load_model, load_image, predict, annotate, Model
import cv2
import matplotlib.pyplot as plt
import numpy as np
from const import *
import supervision as sv
img_path = 'assets/61.jpg'
text_prompt = ['cap', 'shirt', 'sunglasses', 'shoe', 'sock', 'backpack', 'sticks', 'bib', 'trousers']






def detect_with_classes (image_path, text_prompt, box_threshold=0.3, text_threshold=0.3):
    image = cv2.imread(image_path)
    grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_T_CHECKPOINT_PATH, device=DEVICE)

    detections, phrases = grounding_dino_model.predict_with_caption(
        image=image,
        caption=", ".join(text_prompt),
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )
    print (phrases)
    classe_id_list_str =  grounding_dino_model.phrases2classes(phrases, text_prompt)
    detections.class_id = classe_id_list_str
    #detections = detections[detections.class_id != text_prompt.index("trousers")]
    box_annotator = sv.BoxAnnotator()
    annotated_image = box_annotator.annotate(scene=image.copy(), detections=detections, labels=phrases)

    return annotated_image, detections, phrases

annotated_frame, detections, phrases = detect_with_classes(img_path, text_prompt)
print (detections)

cv2.imwrite('assets/61_dino_classes.jpg', annotated_frame)

In [ ]:

import numpy as np
from segment_anything import sam_model_registry, SamPredictor

sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_WEIGHTS_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)


def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

In [ ]:
# convert detections to masks
image = cv2.imread(img_path)
detections.mask = segment(
    sam_predictor=sam_predictor,
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
    xyxy=detections.xyxy
)

# annotate image with detections
box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()
#annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)
annotated_image = box_annotator.annotate(scene=image.copy(), detections=detections, labels=phrases)

sv.plot_image(annotated_image, (16, 16))

In [ ]:
from groundingdino.util.inference import load_model, load_image, predict, annotate, Model
import cv2
import matplotlib.pyplot as plt
import numpy as np
from const import *
import supervision as sv
img_path = 'assets/61.jpg'
text_prompt = ['cap', 'shirt', 'sunglasses', 'shoe', 'sock', 'backpack', 'sticks', 'bib', 'trousers']

from dino_detection import detect_with_dino

annotated_frame, detections, phrases = detect_with_dino(img_path, text_prompt)
print (detections)
sv.plot_image( annotated_image, (16, 16))
cv2.imwrite('assets/61_testdino.jpg', annotated_frame)

In [ ]:


import torch
import supervision as sv
import os
import cv2
import matplotlib.pyplot as plt

from setup import *
from const import *
from dino_detection import detect_with_dino
from segment import segment
import logging
import numpy as np

In [ ]:
print(check_available_weights())
img_path = 'assets/DSC1167.jpg'
text_prompt = ['person']
img = cv2.imread(img_path)
annotated_image, detections, phrases = detect_with_dino(img, text_prompt)

segmented_image, detections2 = segment(detections, img)
# enregistrer les images
cv2.imwrite("assets/segmented_image.jpg", segmented_image)

In [ ]:
#print (detections2)
print (detections2.mask[1])
# save the mask 1

cv2.imwrite("assets/mask1.jpg", detections2.mask[1])



In [ ]:

import math


# plot parameters
grid_size_dimension = math.ceil(math.sqrt(len(detections.mask)))


sv.plot_images_grid(
    images = detections.mask,

    grid_size = (grid_size_dimension, grid_size_dimension),
    size = (16, 16)
)

In [ ]:

for i in range(len(detections.mask)):
    # open the original mask
    mask = detections.mask[i]
    

    # manual binarisation
    for j in range(len(mask)):
        for k in range(len(mask[j])):
            if mask[j][k] :
                mask[j][k] = 255
            else:
                mask[j][k] = 0

    # save the new mask
    mask = mask.astype(np.uint8) * 255
    cv2.imwrite(f"assets/tests/mask{i}.jpg", mask)
   



In [ ]:
from PIL import Image

In [ ]:
def binarize_list_of_masks(list_of_masks):
    binarized_masks = list_of_masks
    for i in range(len(list_of_masks)):
        binarized_masks[i] = binarize_mask(list_of_masks[i])
    return binarized_masks

def binarize_mask(mask):
    binarized_mask= mask
    for j in range(len(mask)):
        for k in range(len(mask[j])):
            if mask[j][k] :
                binarized_mask[j][k] = 255
            else:
                binarized_mask[j][k] = 0
    binarized_mask = binarized_mask.astype(np.uint8) * 255
    return binarized_mask

def plot_masks(list_of_masks):
    
    for i in range(len(list_of_masks)):
        # get mask information
        mask = list_of_masks[i]
        

        # show the mask
        fig, ax = plt.subplots()
        ax.imshow(mask, cmap='gray')
        ax.set_axis_off()
        plt.show()

In [ ]:
plot_masks(binarize_list_of_masks(detections.mask))

In [ ]:
print (binarize_mask(detections.mask[0]))

In [ ]:

def colorize_mask(bin_mask, img):
    
    img_arr = np.asarray(img)
    bin_mask = np.expand_dims(bin_mask, axis=2)
    result_arr = np.bitwise_and(img_arr, bin_mask)
    result_img = Image.fromarray(result_arr)
    #convert to RGB with cv2
    result_img = cv2.cvtColor(np.array(result_img), cv2.COLOR_BGR2RGB)
    return result_img
def colorize_list_of_masks(list_of_masks, img):
    colorized_masks = []
    for i in range(len(list_of_masks)):
        colorized_masks.append(colorize_mask(list_of_masks[i], img))
    return colorized_masks

In [ ]:
plt.imshow(colorize_mask(binarize_mask(detections.mask[0]), img_path))

In [1]:
from process import *
from color_detection import *
from color_classifier   import *
from yolo_detection import detect_objects, concat_bib_numbers, extract_bib_numbers,sort_bib_numbers



c:\Users\cleme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [ ]:
img_path = 'assets/DSC1167.jpg'

img = cv2.imread(img_path)
listA = first_step(img)

In [ ]:
second_step(listA, img_path)

In [ ]:

def step3(img_path):

    img = cv2.imread(img_path)
    text_prompt = ['cap', 'shirt', 'sunglasses', 'shoe', 'sock', 'backpack', 'sticks', 'bib', 'trousers']
    annotated_image, segmented_image, detections, phrases = detect_and_segment(img, text_prompt)

    
    colorized_masks =colorize_list_of_masks(binarize_list_of_masks(detections.mask), img)
    filtered_masks = []
    filtered_phrases = []
    for i in range(len(detections.mask)):
        if phrases not in ['bib', 'sticks']:
            filtered_masks.append(colorized_masks[i])
            filtered_phrases.append(phrases[i])
    #tab_names , average_colors_hexa=determine_color_v2(colorized_masks,phrases)
    tab_names , average_colors_hexa=determine_color_v2(filtered_masks,filtered_phrases)
    return tab_names , average_colors_hexa,detections, phrases,colorized_masks,annotated_image, segmented_image



# verification des doublons de detection
"""
for i in range(len(detections)):
    for j in range(len(detections)):
        if i != j:
            if detections[i].class_id == detections[j].class_id:
                LOGGER.warning(f"double detection for {detections[i].class_id} at {i} and {j}")
                # supprimer la detection le de plus petit score
                if detections[i].score < detections[j].score:
                    detections.remove(detections[i])
                else:
                    detections.remove(detections[j])
"""



In [1]:
from process import *
from color_detection import *
from color_classifier   import *
from yolo_detection import detect_objects, concat_bib_numbers, extract_bib_numbers,sort_bib_numbers


c:\Users\cleme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [2]:
img_p =  'assets/tests/p_2_259399375.jpg'
img = cv2.imread(img_p)
tab_names , average_colors_hexa,detections, phrases, annotated_image,segmented_image,croped_bib , text_prompt= step3(img)
cv2.imwrite('assets/tests/segmented_image.jpg', segmented_image)
cv2.imwrite('assets/tests/annotated_image.jpg', annotated_image)
dict = step5(tab_names , average_colors_hexa, phrases ,croped_bib, text_prompt, "gg")
print (dict)

c:\Users\cleme\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


c:\Users\cleme\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
c:\Users\cleme\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\cleme\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


shirt
cap
trousers
Temps d'exécution : 2.575484275817871 secondes
(223, 0.74, 0.66)
shirt
Temps d'exécution : 1.5374042987823486 secondes
(353, 0.06, 0.73)
cap
Temps d'exécution : 2.815484046936035 secondes
(229, 0.34, 0.29)
trousers
step3 done
shirt :  Blue (223, 0.74, 0.66)
cap :  Grey (353, 0.06, 0.73)
trousers :  Grey (229, 0.34, 0.29)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
{'person': None, 'cap': {'detected': False, 'color': None}, 'shirt': {'detected': False, 'color': None}, 'sunglasses': {'detected': False}, 'shoe': {'detected': False}, 'sock': {'detected': False}, 'backpack': {'detected': False}, 'sticks': {'detected': False}, 'number': {'detected': False, 'numbers': None}, 'trousers': {'detected': False, 'color': None}}
Blue
Grey
Grey
{'person': 'gg', 'cap': {'detected': True, 'color': 'Grey'}, 'shirt': {'detected': True, 'color': 'Blue'}, 'sunglasses': {'detected': False}, 'shoe': {'detected': True}, 'sock'

In [ ]:
print (average_colors_hexa)

In [ ]:
def step42(tab_names , average_colors_hexa,detections, phrases ,colorized_masks):
    # recuperer le mask du bib 
    # si on detecte le bib
    if 'number' in phrases:
        for  mask , label in zip (colorized_masks , phrases):
            if label == 'number':
                mask_bib = mask
                croped_bib = crop_bip_numbers(phrases, mask_bib, detections)
    else:
        croped_bib = None
    return croped_bib

def step52 (tab_names , average_colors_hexa,detections, phrases ,colorized_masks,croped_bib):
    dict_color = {}
    for name , color in zip (tab_names , average_colors_hexa ):
        detected_color = color_or_grayscale(color)
        print(name ,": ", color_or_grayscale(color),color)
        dict_color[name] = detected_color
    if croped_bib is not None:
        detected_objects = detect_objects(croped_bib, device=DEVICE)
        bibs= concat_bib_numbers(extract_bib_numbers(sort_bib_numbers(detected_objects)))
        print (bibs)
        # parcourir les phrases et le dictionnaire
        for phrase in phrases:
            if phrase in dict:
                print ("ok")
                if phrase in ['cap', 'shirt', 'trousers']:
                    print (dict_color[phrase])
                    dict[phrase]['color'] = dict_color[phrase]
                    dict[phrase]['detected'] = True
                elif phrase == 'number':
                    dict[phrase]['detected'] = True
                    dict[phrase]['numbers'] = bibs
                else:
                    dict[phrase]['detected'] = True
        
    return dict

crop_bip=step42(tab_names , average_colors_hexa,detections, phrases,colorized_masks)

print (step52(tab_names , average_colors_hexa,detections, phrases ,colorized_masks,crop_bip))

In [ ]:
def step5 (tab_names , average_colors_hexa,detections, phrases ,colorized_masks,croped_bib):
    dict_color = {}
    for name , color in zip (tab_names , average_colors_hexa ):
        detected_color = color_or_grayscale(color)
    #print(name ,": ", color_or_grayscale(color),color)
        dict_color[name] = detected_color
    # si on detecte le bib

    print (dict_color)
    if croped_bib is not None:
        detected_objects = detect_objects(croped_bib, device=DEVICE)
        bibs= concat_bib_numbers(extract_bib_numbers(sort_bib_numbers(detected_objects)))
    for detection in dict_color :
        if detection in dict:
            dict[detection]['detected'] = True
            if detection != 'bib':
                dict[detection]['color'] = dict_color[detection]
            else:
                dict[detection]['numbers'] = bibs
        else:
            dict[detection]['detected'] = False
    return dict
print (step5(tab_names , average_colors_hexa,detections, phrases ,colorized_masks,crop_bip))

In [ ]:

cv2.imwrite('assets/tests/p_0_259399375_dino.jpg', annotated_image)
cv2.imwrite('assets/tests/p_0_259399375_segmented.jpg', segmented_image)

In [ ]:

# TODO: supprimer affichage des couleurs

print (dict_color)


In [ ]:

def get_bib_numbers_index(tab_labels):
    index = 0
    for label in tab_labels:
        if label == 'bib':
            return index
        index += 1
    return None

def crop_bip_numbers(tab_labels,image):
    index= get_bib_numbers_index(tab_labels)
    if index is not None:
        # rounding the float numbers
        x1 = math.floor(detections.xyxy[index][0])
        y1 = math.floor(detections.xyxy[index][1])
        x2 = math.floor(detections.xyxy[index][2])
        y2 = math.floor(detections.xyxy[index][3])
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        # crop the original image with the detection box
        cropped_image = image.crop((x1, y1, x2, y2))
        return cropped_image
    else:
        print("any bip numbers found")
        return None
    

croped_bib = crop_bip_numbers(phrases, img)

if croped_bib is not None:
    croped_bib.save("assets/tests/bib.jpg")
    print("bib numbers saved")


In [ ]:
import sys
from pathlib import Path

# Spécifiez le chemin complet de votre répertoire YOLOv7
yolov7_path = Path('./yolov7')
sys.path.append(str(yolov7_path.resolve()))
from yolov7.detect import detect

In [ ]:

import cv2
source_path = "./assets/tests/bib.jpg" # Remplacez cela par votre chemin d'accès à l'image ou à la vidéo
img = cv2.imread(source_path)
detected_objects = detect_objects(img)


for detection in detected_objects:
    print(f"Class: {detection['class']}, Label: {detection['label']}, Confidence: {detection['confidence']}, Box: {detection['box']}")


In [ ]:


# afficher les numéros de dossard
print(concat_bib_numbers(extract_bib_numbers(sort_bib_numbers(detected_objects))))



In [ ]:
# remplissage du dictionnaire
# pour chaque objet  on verifie si il est detecté
# si oui on rempli la couleur ou le numero de dossard
for detection in dict_color :
    if detection in dict:
        dict[detection]['detected'] = True
        if detection != 'bib':
            dict[detection]['color'] = dict_color[detection]
        else:
            dict[detection]['numbers'] = concat_bib_numbers(extract_bib_numbers(sort_bib_numbers(detected_objects)))
    else:
        dict[detection]['detected'] = False

print (dict)

In [ ]:
text_prompt = ['person']
annotated_image, segmented_image, detections, phrases = detect_and_segment(img, text_prompt)

binarized_list_of_masks=binarize_list_of_masks(detections.mask)
colorized_list_of_masks=colorize_list_of_masks(binarized_list_of_masks, img)

In [ ]:
plt.imshow( binarized_list_of_masks[0])
print (binarize_mask(detections.mask[0]))
print (binarized_list_of_masks[0])


In [ ]:
second_step(colorized_list_of_masks)